# Open Knowledge Graph Tutorial

This short notebook will run through an example use case of  `belief_graph` for automatic construction of an open knowledge graph. Some of the main functionalities of `belief_graph` will be covered, but this tutorial is not extensive. 

We will use the Twitter part of the [Danish Gigaword Corpus](https://gigaword.dk) for this exploratory analysis. 

In [3]:
import sys
sys.path.append('../')

import belief_graph as bg

In [ ]:
# Load data and extract text to list

In [ ]:
# First off, let's load the Spacy nlp pipeline

# load_danish() is a helper method that loads the Danish_news_sm pipeline with the Danish Electra transformer model
nlp = bg.load_danish()

# Now, instantiate the BeliefParser which parses the texts and returns BeliefTriplets. These consist of a head word(s), tail word(s), and their relation (e.g. h: Mette Frederiksen, r: er, t: statsminister)
bp = bg.BeliefParser(nlp=nlp)

# BeliefGraph is used to gather the BeliefTriplets into a graph. Once in the graph, we can do all sorts of interesting network analysis, or plot specific nodes and its relations
graph = bg.BeliefGraph(parser=bp)

Now with everything set up, we are ready to add texts to the pipeline.

In [ ]:
graph.add_texts(tweet_list)


The relations we have extracted now are very noisy - let's see an example.

In [ ]:
graph.plot_node(nodes = "Mette Frederiksen")

To handle this, `belief_graph` includes the following filters:

- ConfidenceFilter: filter by how confident the model is   
- ContinuousFilter: only includes words that appear continuously
- CountFilter: require nodes to appear a certain number of times in the corpus
- DepFilter: include/exclude specific terms from a dependency parsing
- EntFilter: include/exclude specific named entities (LOC, PERS, ORG, MISC in Danish)
- PosFilter: include/exclude specific Parts-of-Speech tags
- LemmatizationFilter: use lemmas instead of tokens

These can be mixed and matched as you see fit. Let's see the effect of setting a confidence threshold and only including nodes that have the LOC or PERS tag.

In [ ]:
CONF_THRESHOLD = 0.85
conf_filter = bg.filters.ConfidenceFilter(threshold=CONF_THRESHOLD)

VALID_ENTITIES={"LOC", "PERS"}
ent_filter = bg.filters.EntFilter(valid=VALID_ENTITIES)

# To update the graph we pass the filters as a list
filters = [conf_filter, ent_filter]

graph.replace_filters(triplet_filters=filters)

# Let's see how the network has changed
graph.plot_node(nodes = "Mette Frederiksen")

You can plot networks for multiple nodes by passing a list of node names to the `nodes` parameter in `graph.plot_node`. Similarly, it is easy to change filters: simply create the ones you want, and run `graph.replace_filters` with your updated filter list. 

If you pass the keyword "all" to `nodes` the network will include all nodes in the graph. This is especially useful in combination with `return_graph=True`, as this returns the networkx graph. With this, you can explore the entire network and calculate various measures such as centrality, degree, etc.

In [ ]:
network = graph.plot_node(nodes = "all", return_graph=True)

network.G